In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm, trange

storage = "/Volumes/easystore/Drones/"

cv_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_cv-flight-manifest.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
per_second = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_drone-seconds-block-count.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)


per_day = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_cv-blocks-drones-calls.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
race_df = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_census-race.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
cv_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_cv-blocks-geometry.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)


# flight_data = pd.read_csv("/Volumes/easystore/Drones/compiled-flight-data-gpx.csv",dtype=str)
# flight_data.head()

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [3]:
per_second["seconds"] = per_second["seconds"].astype(float)
per_second["call_count"] = per_second["call_count"].astype(float)

per_day["num_flights"] = per_day["num_flights"].astype(float)
per_day["num_calls"] = per_day["num_calls"].astype(float)

In [3]:
per_second_joined_race = pd.merge(per_second, race_df, on="GEOID20", how="left")

In [6]:
per_second_joined_race[['Total','Total Alone', 'Total Latino',
       'Total Not Latino', 'Total White']]

,Total,Total Alone,Total Latino,Total Not Latino,Total White
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,655,188,459,196,70
4,93,16,75,18,15
...,...,...,...,...,...
1954,0,0,0,0,0
1955,0,0,0,0,0
1956,21,2,15,6,2
1957,574,272,270,304,82


In [4]:
per_second_joined_race["latino_exposure"] = per_second_joined_race[
    "Total Latino"
].astype(int) * per_second_joined_race["seconds"].astype(float)

per_second_joined_race["not_latino_exposure"] = per_second_joined_race[
    "Total Not Latino"
].astype(int) * per_second_joined_race["seconds"].astype(float)

In [11]:
(
    (
        per_second_joined_race["latino_exposure"].sum()
        / per_second_joined_race["Total Latino"].astype(int).sum()
    )
    / 60
) / 60

3.789477439801444

In [18]:
per_second_joined_race["white_exposure"] = per_second_joined_race[
    "Total Black or African American"
].astype(int) * per_second_joined_race["seconds"].astype(float)

In [19]:
per_second_joined_race["white_exposure"].sum() / per_second_joined_race[
    "Total Black or African American"
].astype(int).sum()

12250.488063137125

In [6]:
per_second_joined_race["not_latino_exposure"].sum() / per_second_joined_race[
    "Total Not Latino"
].astype(int).sum()

8441.519710057777

In [7]:
13642.118783285197 / 8441.519710057777

1.6160737938017353

In [9]:
coi = [
    "GEOID20",
    "NAME20",
    "MTFCC20",
    "ALAND20",
    "AWATER20",
    "INTPTLAT20",
    "INTPTLON20",
    "HOUSING20",
    "POP20",
    "geometry",
    "seconds",
    "weight",
    "call_count",
    "Geography",
    "Total",
    "White",
    "Black or African American",
    "American Indian and Alaska Native",
    "Asian",
    "Native Hawaiian and Other Pacific Islander",
    "Some Other Race",
    "Two or More Races",
]

In [10]:
per_second_joined_race

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,Total Alone,Total Latino,Total Not Latino,Total White,Total Black or African American,Total American Indian and Alaska Native,Total Asian,Total Native Hawaiian and Other Pacific Islander,Total Some Other Race,len_y
0,6,73,12600,1005,060730126001005,Block 1005,G5040,U,78661.0,U,...,0,0,0,0,0,0,0,0,0,15
1,6,73,12600,1000,060730126001000,Block 1000,G5040,U,78661.0,U,...,0,0,0,0,0,0,0,0,0,15
2,6,73,12600,1001,060730126001001,Block 1001,G5040,U,78661.0,U,...,0,0,0,0,0,0,0,0,0,15
3,6,73,12600,1002,060730126001002,Block 1002,G5040,U,78661.0,U,...,188,459,196,70,51,1,66,0,0,15
4,6,73,12600,1003,060730126001003,Block 1003,G5040,U,78661.0,U,...,16,75,18,15,0,0,0,1,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1954,6,73,10016,1000,060730100161000,Block 1000,G5040,R,NaN,NaN,...,0,0,0,0,0,0,0,0,0,15
1955,6,73,10016,1002,060730100161002,Block 1002,G5040,R,NaN,NaN,...,0,0,0,0,0,0,0,0,0,15
1956,6,73,13322,3005,060730133223005,Block 3005,G5040,U,78661.0,U,...,2,15,6,2,0,0,0,0,0,15
1957,6,73,13322,3000,060730133223000,Block 3000,G5040,U,78661.0,U,...,272,270,304,82,39,0,148,3,0,15


# Lots of Flights

In [21]:
per_day

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,INTPTLON20,HOUSING20,POP20,geometry,len,num_flights,flights_per_day,num_calls,calls_per_day,delta
0,6,73,12600,1005,060730126001005,Block 1005,G5040,U,78661.0,U,...,-117.0972764,0,1,"POLYGON ((-117.09848 32.636082, -117.098118 32...",15,163.0,0.2058080808080808,0.0,0.0,163.0
1,6,73,12600,1000,060730126001000,Block 1000,G5040,U,78661.0,U,...,-117.0968738,0,1,"POLYGON ((-117.098419 32.636099, -117.098382 3...",15,207.0,0.26136363636363635,0.0,0.0,207.0
2,6,73,12600,1001,060730126001001,Block 1001,G5040,U,78661.0,U,...,-117.0948246,0,1,"POLYGON ((-117.095692 32.629235, -117.095466 3...",15,131.0,0.16540404040404041,0.0,0.0,131.0
3,6,73,12600,1002,060730126001002,Block 1002,G5040,U,78661.0,U,...,-117.0932719,310,656,"POLYGON ((-117.095251 32.629345, -117.095177 3...",15,241.0,0.3042929292929293,1170.0,1.4772727272727273,-929.0
4,6,73,12600,1003,060730126001003,Block 1003,G5040,U,78661.0,U,...,-117.0915697,39,94,"POLYGON ((-117.092611 32.630074, -117.092519 3...",15,221.0,0.27904040404040403,28.0,0.03535353535353535,193.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1954,6,73,10016,1000,060730100161000,Block 1000,G5040,R,NaN,NaN,...,-116.9514464,0,1,"POLYGON ((-116.966156 32.589925, -116.965916 3...",15,1.0,0.0012626262626262627,0.0,0.0,1.0
1955,6,73,10016,1002,060730100161002,Block 1002,G5040,R,NaN,NaN,...,-116.9902654,0,1,"POLYGON ((-117.005217 32.592376, -117.005127 3...",15,2.0,0.0025252525252525255,0.0,0.0,2.0
1956,6,73,13322,3005,060730133223005,Block 3005,G5040,U,78661.0,U,...,-116.9924214,4,22,"POLYGON ((-116.993114 32.628806, -116.993053 3...",15,4.0,0.005050505050505051,8.0,0.010101010101010102,-4.0
1957,6,73,13322,3000,060730133223000,Block 3000,G5040,U,78661.0,U,...,-116.991596,162,575,"POLYGON ((-116.996325 32.633525, -116.996153 3...",15,56.0,0.0707070707070707,238.0,0.3005050505050505,-182.0


In [19]:
calls_for_service.sort_values("call_count").tail(10)[["GEOID20"]]

,GEOID20
1586,060730125022001
367,060730131032011
1585,060730126001004
1920,060730127003000
1111,060730125011000
691,060730133232000
1196,060730131032012
1317,060730134144005
1078,060730132051000
340,060730133214002
